# 0. Description
1. 考虑一个三变量函数，坐标区间是`[-1, 1]`。寻找这个函数在`(0.5, 0.5, 0.5)` 和 `(-0.5, -0.5, -0.5)` 附近的两个局部最小值，以及`两个极小值之间最低能量路径上的鞍点`。

In [1]:
import numpy as np
from scipy.optimize import minimize

In [2]:
def gaussian(pos:np.array, pos0:np.array):
    '''
    Description
    -----------
        1. Meassure the distance between pos and pos0
    
    Parameters
    ----------
        1. pos: np.array (一维)
        2. pos0: np.array (一维)
    '''
    return np.exp( -np.sum(np.power(pos-pos0, 2), axis=0) )


def data(pos:np.array):
    return gaussian(pos, np.array([0.1, -0.1, -0.1])) - \
            gaussian(pos, np.array([-0.5, -0.5, -0.5])) - \
            gaussian(pos, np.array([0.5, 0.5, 0.5]))

# 1. `BFGS` 算法求解 `(0.5, 0.5, 0.5)`, `(-0.5, -0.5, -0.5)` 附近的最小值

In [4]:
pos_1 = [0.5, 0.5, 0.5]
res_1 = minimize(fun=data, x0=pos_1, method="BFGS")
print("Point 1: ", res_1.x)

pos_2 = [-0.5, -0.5, -0.5]
res_2 = minimize(fun=data, x0=pos_2, method="BFGS")
print("Point 2: ", res_2.x)

Point 1:  [0.6052717  0.67115817 0.67115817]
Point 2:  [-0.73110579 -0.64589743 -0.64589743]


# 2. `Target function` 的梯度
1. 根据定义直接写

In [5]:
def gradient(pos, delta = 0.01):
    '''
    Desription
    ----------
        1. 计算 image 在三个方向的梯度
    
    Parameters
    ----------
        1. pos: np.array (一维)
            image 的坐标
        2. delta: float
            每次的位移量
    '''
    gradient_x = ( data(pos + np.array([delta, 0, 0])) - data(pos - np.array([delta, 0, 0])) ) / (2 * delta)
    gradient_y = ( data(pos + np.array([0, delta, 0])) - data(pos - np.array([0, delta, 0])) ) / (2 * delta)
    gradient_z = ( data(pos + np.array([0, 0, delta])) - data(pos - np.array([0, 0, delta])) ) / (2 * delta)

    return np.array([gradient_x, gradient_y, gradient_z])

# 3. 寻找鞍点

## 3.1. 初始化 `images` -- 直接等间距插点

In [8]:
# 1. NEB 的 start_point 和 end_point
pos_A = np.array([0.6052717, 0.67115817, 0.67115817])
pos_B = np.array([-0.73110579, -0.64589743, -0.64589743])

# 2. NEB images的数量（包含 start_point 和 end_point）
num_images = 11

# 3. 初始化所有 images 的坐标
images = np.zeros([num_images, 3])

for i in range(num_images):
    images[i] = (pos_B - pos_A) / (num_images-1) * i + pos_A

print(images)

[[ 0.6052717   0.67115817  0.67115817]
 [ 0.47163395  0.53945261  0.53945261]
 [ 0.3379962   0.40774705  0.40774705]
 [ 0.20435845  0.27604149  0.27604149]
 [ 0.0707207   0.14433593  0.14433593]
 [-0.06291704  0.01263037  0.01263037]
 [-0.19655479 -0.11907519 -0.11907519]
 [-0.33019254 -0.25078075 -0.25078075]
 [-0.46383029 -0.38248631 -0.38248631]
 [-0.59746804 -0.51419187 -0.51419187]
 [-0.73110579 -0.64589743 -0.64589743]]


## 3.2. 定义`弹力的合力`的 `大小` 和 `方向`

In [11]:
# start_point 和 end_point 之间的距离
dist_AB = np.sqrt( np.sum( np.power(pos_A - pos_B, 2) , axis=0) )
# 相邻 images 之间的距离 (弹簧的原长，基于胡克定律计算弹簧力)
dist_image = dist_AB / (num_images - 1)

# 计算弹簧合力的大小和方向
def spring_force(image_before: np.array, 
                image: np.array,
                image_next: np.array,
                k = 2.0,
                ):
    '''
    Description
    -----------
        1. 计算弹簧合力的大小和方向
    
    Parameters
    ----------
        1. image_before: np.array (一维)
            image 的坐标 
        2. image: np.array (一维)
            image 的坐标 
        3. image_next: np.array (一维)
            image 的坐标 
        4. k: float
            弹簧常数
    '''
    dist_before = np.sqrt( np.sum(np.power(image - image_before, 2), axis=0) )
    # 胡克定律
    force_before = (dist_before - dist_image) * k
    # 单位方向向量
    direction_before = (image - image_before) / dist_before

    dist_next = np.sqrt( np.sum(np.power(image_next - image, 2), axis=0) )
    # 胡克定律
    force_next = (dist_image - dist_next) * k
    # 单位方向向量
    direction_next = (image_next - image) / dist_next

    ### 计算两根弹簧的合力
    force = force_before * direction_before + force_next * direction_next
    ### 从 image_before 指向 image_next 的方向向量
    direction = (image_next - image_before) / np.sqrt( np.sum(np.power(image_next - image_before, 2), axis=0) )

    return force, direction

## 3.3. 采用`弹力 (平行于轨迹)`和`重力 (垂直于轨迹)`的合力进行迭代
1. 判断条件：
   1. 前后两次迭代的`最大坐标差`
   2. 前后两次迭代的`最大Target Function差`

In [18]:
# 等间距插入 images
for i in range(num_images):
    images[i] = (pos_B - pos_A) / (num_images - 1) * i + pos_A

# 初始化弹簧力、重力、方向(image_next - image_before)数组
s_forces = np.zeros_like(images)
directions = np.zeros_like(images)
g_forces = np.zeros_like(images)

# image 位置更新速率
rate = 0.1
# NEB 收敛判据
err = 1e-8

# NEB driver code
def NEB(rate:float, err:float):
    n_step = 0
    pos_diff = 1    # pos_diff: 前后两次迭代的坐标差的最大值（某个方向, float）
    data_diff = 1   # data_diff: 前后两次迭代的最大Target Function差的绝对值

    while (pos_diff > err) or (data_diff > err):
        # old_pos: 所有 images 的坐标
        old_pos = images
        # old_saddle: 所有 images 对应的最大的 Target function
        old_saddle = np.max([data(images[i]) for i in range(num_images)])
        
        for i in range(1, num_images-1):
            s_forces[i], directions[i] = spring_force(image_before=images[i-1],
                                                    image=images[i],
                                                    image_next=images[i+1],
                                                    )
            # 只需要弹簧力沿着轨迹的部分, Vector Decomposition
            s_forces[i] = np.dot(s_forces[i], directions[i]) * directions[i]
            
            g_forces[i] = gradient(images[i])
            # 只需要重力垂直于轨迹的部分, Vector Decomposition
            g_forces[i] = g_forces[i] - np.dot(g_forces[i], directions[i]) * directions[i]
            
            # 更新 image 的坐标 
            images[i] -= (s_forces[i] + g_forces[i]) * rate
        
        # new_pos: 更新后的 pos
        new_pos = images
        # new_saddle: 最大的 Target Function
        new_saddle = np.max([data(images[i]) for i in range(num_images)])
        # idx_saddle: 最大Target function 对应的 image 的索引
        idx_saddle = np.argmax([data(images[i]) for i in range(num_images)])

        pos_diff = np.max(np.abs(new_pos - old_pos))
        data_diff = np.abs(new_saddle - old_saddle)
        n_step += 1
    
    return n_step, idx_saddle

In [21]:
n_step, idx_saddle = NEB(rate, err)
print("n_step    ", n_step)
print("idx_saddle", idx_saddle)
print("images    ", images[idx_saddle])
print("data      ", data(images[idx_saddle]))
print("gradient  ", gradient(images[idx_saddle]))  
### Note: 中间构型仍然存在沿着最低能量路径方向的梯度，说明其并不在鞍点位置。我们需要进一步精修其位置

n_step     1
idx_saddle 5
images     [-0.59593054  0.28021293  0.28021293]
data       -0.10501252488584867
gradient   [-0.01279937 -0.01331384 -0.01331384]


# 4. Climbing Image Nudged Elastic Band
1. 采用`重力 (沿着轨迹的方向)`更新位置，判断条件是珠子受重力方向的最大值

In [23]:
def cNEB(image:np.array,
        direction:np.array):
    '''
    Parameters
    ----------
        1. image: np.array (一维)
            image 的坐标
        2. direction: np.array (一维)
            image 的轨迹方向 (image_next - image_before)
    '''
    g_force = 1
    while np.max(np.abs(g_force)) > err:
        g_force = gradient(image)
        g_force = np.dot(g_force, direction) * direction
        image += g_force * rate
    return image

In [25]:
saddle_direction = (images[idx_saddle+1] - images[idx_saddle-1]) / \
                    np.sqrt( np.sum(np.power(images[idx_saddle+1] - images[idx_saddle-1], 2), axis=0) )
saddle_point = cNEB(image=images[idx_saddle],
                    direction=saddle_direction,
                    )

print("saddle point   ", saddle_point)
print("gradient       ", gradient(saddle_point))
print("target function", data(saddle_point))


# 精度还是不错的，梯度非常接近零啦

saddle point    [-0.60496254  0.27145535  0.27145535]
gradient        [ 0.00056612 -0.00029194 -0.00029194]
target function -0.10483814484384196
